In [1]:
from nuscenes import NuScenes
from nuscenes.eval.common.data_classes import EvalBoxes
from nuscenes.eval.detection.data_classes import DetectionBox 
from pyquaternion import Quaternion
import json
from tqdm import tqdm

In [6]:
detection_file = '/home/mqdao/Downloads/nuScene/detection-megvii/centerPoint_test.json'
data_root = '/home/mqdao/Downloads/nuScene/v1.0-test'
version = 'v1.0-test'

In [7]:
nusc = NuScenes(version=version, dataroot=data_root, verbose=True)

Loading NuScenes tables for version v1.0-test...
23 category,
8 attribute,
4 visibility,
0 instance,
12 sensor,
1800 calibrated_sensor,
462901 ego_pose,
15 log,
150 scene,
6008 sample,
462901 sample_data,
0 sample_annotation,
4 map,
Done loading in 4.2 seconds.
Reverse indexing ...
Done reverse indexing in 0.7 seconds.


In [4]:
# load detection
with open(detection_file) as f:
    data = json.load(f)
all_results = EvalBoxes.deserialize(data['results'], DetectionBox)
meta = data['meta']
print('meta: ', meta)
print("Loaded results from {}. Found detections for {} samples.".format(
    detection_file,
    len(all_results.sample_tokens)))

meta:  {'use_camera': False, 'use_lidar': True, 'use_radar': False, 'use_map': False, 'use_external': False}
Loaded results from /home/mqdao/Downloads/nuScene/detection-megvii/centerPoint_test.json. Found detections for 6008 samples.


In [8]:
# get all scene tokens in validation set
processed_scene_tokens = set()
for sample_token_idx in tqdm(range(len(all_results.sample_tokens))):
    sample_token = all_results.sample_tokens[sample_token_idx]
    scene_token = nusc.get('sample', sample_token)['scene_token']
    if scene_token in processed_scene_tokens:
        continue
    processed_scene_tokens.add(scene_token)

100%|██████████| 6008/6008 [00:00<00:00, 10269.68it/s]


In [9]:
len(processed_scene_tokens)

150

In [10]:
NUSCENES_TRACKING_NAMES = [
  'bicycle',
  'bus',
  'car',
  'motorcycle',
  'pedestrian',
  'trailer',
  'truck'
]
score_threshold = 0.2

In [12]:
# extract detection for each scene
for scene_token in tqdm(processed_scene_tokens):
    all_measurements = {}
    all_classes = {}
    current_time_step = 0
    scene_record = nusc.get('scene', scene_token)
    sample_token = scene_record['first_sample_token']
    while sample_token != '':
        sample_detections = [detection for detection in all_results.boxes[sample_token]
                             if detection.detection_score > score_threshold]
        all_measurements[current_time_step] = []
        all_classes[current_time_step] = []
        for detection in sample_detections:
            # get obj_type
            if detection.detection_name not in NUSCENES_TRACKING_NAMES:
                continue
            obj_type = detection.detection_name
            # get object pose
            x, y, z = detection.translation
            quaternion = Quaternion(detection.rotation)
            yaw = quaternion.angle if quaternion.axis[2] > 0 else -quaternion.angle 
            # get size
            size = list(detection.size)
            # get score
            score = detection.detection_score
            # store mearurement & class
            all_measurements[current_time_step].append([x, y, yaw] + size + [z]+[score])
            all_classes[current_time_step].append(obj_type)
        
        # move on to next sample
        sample_token = nusc.get('sample', sample_token)['next']
        current_time_step += 1
    # save this scene detection in a separate file
    outdata = {'all_measurements': all_measurements,
               'all_classes': all_classes}
    with open('../centerPoint-per-scene-detection/test/{}.json'.format(scene_token), 'w') as outfile:
        json.dump(outdata, outfile)

100%|██████████| 150/150 [00:08<00:00, 18.22it/s]


In [14]:
test_scene_tokens = {}
for i, scene_token in enumerate(processed_scene_tokens):
    test_scene_tokens[i] = scene_token

with open('../test_scene_tokens_centerPoint.json', 'w') as outfile:
    json.dump(test_scene_tokens, outfile)